In [1]:
print("hii nivash")

hii nivash


#froward pass

In [ ]:
import math

class value:
    def __init__(self, data, children = (), op = ''):
        self.data = data
        self.op = op
        self.prev = set(children)
        self._backward = lambda : None
        self.grad = 0.0
        
    def __repr__(self):
        # return f"value is :{self.data}  gradient is : {self.grad} operation is :  {self.op} previous values: {self.prev}\n backward function : {self.backward}\n"
        return f"value({self.data}), grad is : {self.grad}"
    
    def __add__(self, other):
        
        result = value(self.data + other.data, (self, other), '+')
        def grad_calculator():
            self.grad += result.grad
            other.grad += result.grad
        result._backward = grad_calculator 
        return result
    
    def __sub__(self, other):
        
        result = value((self.data - other.data), (self, other), '-')
        
        def grad_calculator():
            self.grad += result.grad * 1.0
            other.grad += result.grad * -1.0
        result._backward = grad_calculator
        return result
        
        
    def __pow__(self, other):
        result = value(self.data**other.data, (self, other), '**')
        def grad_calculator():
            self.grad += other.data * (self.data** (other.data -1)) * result.grad
            other.grad += (self.data**other.data)* math.log(self.data) * result.grad
        result._backward = grad_calculator
        return result
        
    # def __truediv__(self, other):
        
        
    def exp(self):
        result = value(math.exp(self.data), (self,), 'exp')
        def grad_calculator():
            self.grad = result.grad * math.exp(self.data)
        result._backward = grad_calculator
        return result
        
    
    def __mul__(self, other):
        result = value(self.data * other.data, (self, other) , '*')
        def grad_calculator():
            self.grad += result.grad * other.data
            other.grad += result.grad * self.data
        result._backward = grad_calculator
        
        return result
    
    # def __rmul__(self, other):
        
        
        
    

    
    def tanh(self):
        n = self.data
        out = (math.exp(2.0*n) - 1)/(math.exp(2.0*n) + 1)
        result = value(out, (self,), 'tanh')
        def grad_calculator():
            self.grad += 1 - out**2
            
            
        result._backward = grad_calculator
            
        
        return result
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v.prev:
                    build_topo(child)
                topo.append(v)
            
        build_topo(self)
        topo.reverse()
        self.grad = 1.0
        
        
        for x in topo:
            x._backward()
            
        
                    

In [108]:
x = value(3.0)
y = value(4.0)
z = x**y
w = x-1

print(w)

value(2.0), grad is : 0.0


In [102]:
x = value(3.0)
dir(x)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__pow__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_backward',
 'backward',
 'data',
 'exp',
 'grad',
 'op',
 'prev',
 'tanh']

In [78]:
a = value(0.4)
b= value(0.8)
e= a*b
c = value(1.0)
d = c+e
f = value(-1.5)
l = d*f
o = l.tanh()

o.backward()

print("a is : " ,a, "\nb is : ",b, "\nc is :",c, "\nd is : ",d, "\ne is : ",e, "\nf is : ",f,"\nl is : ",l, "\no is : ", o )



a is :  value(0.4), grad is : -0.08811156691047467 
b is :  value(0.8), grad is : -0.044055783455237335 
c is : value(1.0), grad is : -0.11013945863809332 
d is :  value(1.32), grad is : -0.11013945863809332 
e is :  value(0.32000000000000006), grad is : -0.11013945863809332 
f is :  value(-1.5), grad is : 0.09692272360152213 
l is :  value(-1.98), grad is : 0.07342630575872888 
o is :  value(-0.9625869800912908), grad is : 1.0


In [67]:
a = value(0.4)
b= value(0.8)
e= a*b
c = value(1.0)
d = c+e
f = value(-1.5)
l = d*f

m = 2*l
e = l.exp(2*m)

o = (e-1)/(e+1)

o.backward()

print("a is : " ,a, "\nb is : ",b, "\nc is :",c, "\nd is : ",d, "\ne is : ",e, "\nf is : ",f,"\nl is : ",l, "\no is : ", o )


TypeError: unsupported operand type(s) for *: 'int' and 'value'

In [70]:
dir(int)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_count',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'is_integer',
 